우선 훈련을 위한 깃헙 저장소를 다운로드합니다.

In [1]:
print("Download Example Repository")
!git clone https://github.com/jetsonai/DeepLearning4Projects

Download Example Repository
Cloning into 'DeepLearning4Projects'...
remote: Enumerating objects: 1268, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 1268 (delta 220), reused 130 (delta 130), pack-reused 989 (from 1)
Receiving objects: 100% (1268/1268), 163.71 MiB | 14.34 MiB/s, done.
Resolving deltas: 100% (727/727), done.
Updating files: 100% (120/120), done.


그 후 데이터셋을 다운로드합니다.

다운로드에 약 7분 정도의 시간이 필요합니다.

In [5]:
print("Download Dataset")
!git clone https://github.com/jetsonai/Recycle_Classification_Dataset.git
!rm -rf ./Recycle_Classification_Dataset/.git

Download Dataset
Cloning into 'Recycle_Classification_Dataset'...
remote: Enumerating objects: 9442, done.
remote: Total 9442 (delta 0), reused 0 (delta 0), pack-reused 9442 (from 1)
Receiving objects: 100% (9442/9442), 6.87 GiB | 35.83 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (9417/9417), done.


In [ ]:
import os
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm

from .Model_Class_From_the_Scratch import MODEL_From_Scratch
from .Model_Class_Transfer_Learning_MobileNet import MobileNet
from .Dataset_Class import PyTorch_Classification_Dataset_Class as Dataset

class PyTorch_Classification_Training_Class():
    def __init__(self
                , dataset_dir = "/content/Recycle_Classification_Dataset"
                , batch_size = 16
                , train_ratio = 0.75
                ):
        if not os.path.isdir(dataset_dir):
            os.system("git clone https://github.com/JinFree/Recycle_Classification_Dataset.git")
            os.system("rm -rf ./Recycle_Classification_Dataset/.git")
            dataset_dir = os.path.join(os.getcwd(), 'Recycle_Classification_Dataset')        
        self.USE_CUDA = torch.cuda.is_available()
        self.DEVICE = torch.device("cuda" if self.USE_CUDA else "cpu")
        self.transform = transforms.Compose([
                transforms.Resize(256)
                , transforms.RandomCrop(224)
                , transforms.ToTensor()
                , transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]) 
                ])
        dataset = Dataset(dataset_dir = dataset_dir, transform = self.transform)
        dataset.__save_label_map__()
        self.num_classes = dataset.__num_classes__()
        train_size = int(train_ratio * len(dataset))
        test_size = len(dataset) - train_size
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
        self.train_loader = torch.utils.data.DataLoader(
            train_dataset
            , batch_size=batch_size
            , shuffle=True
        )
        self.test_loader = torch.utils.data.DataLoader(
            test_dataset
            , batch_size=batch_size
            , shuffle=False
        )
        self.model = None
        self.model_str = None
        
    def prepare_network(self
            , is_scratch = True):
        if is_scratch:
            self.model = MODEL_From_Scratch(self.num_classes)
            self.model_str = "PyTorch_Training_From_Scratch"
        else:
            self.model = MobileNet(self.num_classes)
            self.model_str = "PyTorch_Transfer_Learning_MobileNet"
        self.model.to(self.DEVICE)
        self.model_str += ".pt" 
    
    def training_network(self
            , learning_rate = 0.0001
            , epochs = 10
            , step_size = 3
            , gamma = 0.3):
        if self.model is None:
            self.prepare_network(False)
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
        acc = 0.0
        for epoch in range(1, epochs + 1):
            self.model.train()
            for data, target in tqdm(self.train_loader):
                data, target = data.to(self.DEVICE), target.to(self.DEVICE)
                optimizer.zero_grad()
                output = self.model(data)
                loss = F.cross_entropy(output, target)
                loss.backward()
                optimizer.step()
            scheduler.step()
            self.model.eval()
            test_loss = 0
            correct = 0
            with torch.no_grad():
                for data, target in tqdm(self.test_loader):
                    data, target = data.to(self.DEVICE), target.to(self.DEVICE)
                    output = self.model(data)
                    test_loss += F.cross_entropy(output, target, reduction='sum').item()
                    pred = output.max(1, keepdim=True)[1]
                    correct += pred.eq(target.view_as(pred)).sum().item()
            test_loss /= len(self.test_loader.dataset)
            test_accuracy = 100. * correct / len(self.test_loader.dataset)
            print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))
            if acc < test_accuracy or epoch == epochs:
                acc = test_accuracy
                torch.save(self.model.state_dict(), self.model_str)
                print("model saved!")
        
if __name__ == "__main__":
    training_class = PyTorch_Classification_Training_Class()
    training_class.prepare_network(True)
    training_class.training_network()


In [ ]:
#from .Model_Class_From_the_Scratch import MODEL_From_Scratch
import torch
import torch.nn as nn
import torch.nn.functional as F

class PyTorch_Custom_Model_Class(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    
    def forward(self, x):
        return x

class MODEL_From_Scratch(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 2, padding = 1)
            , nn.BatchNorm2d(32)
            , nn.ReLU()
            , nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1)
            , nn.BatchNorm2d(64)
            , nn.ReLU()
            , nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1)
            , nn.BatchNorm2d(128)
            , nn.ReLU()
            , nn.AdaptiveAvgPool2d(1)
            , nn.Flatten()
            , nn.Linear(128, 512)
            , nn.ReLU()
            , nn.Dropout()
            , nn.Linear(512, 64)
            , nn.ReLU()
            , nn.Dropout()
            , nn.Linear(64, num_classes)
            , nn.Softmax(dim=-1)
        )
    def forward(self, x):
        return self.classifier(x)

# from .Model_Class_Transfer_Learning_MobileNet import MobileNet
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

class MobileNet(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        self.network = models.mobilenet_v2(pretrained=pretrained)
        num_ftrs = self.network.classifier[1].in_features 
        self.network.classifier[1] = nn.Linear(num_ftrs, num_classes)
        self.classifier = nn.Sequential(nn.Softmax(dim=-1)) 

    def forward(self, x):
        x = self.network(x)  
        x = self.classifier(x) 
        return x

훈련을 위한 클래스를 불러옵니다.

In [6]:
from DeepLearning4Projects.Chap5.Training_Class import PyTorch_Classification_Training_Class
training_class = PyTorch_Classification_Training_Class()

/content/DeepLearning4Projects/Chap5/Dataset_Class.py:50: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if image.mode is not "RGB":


우선 임의 제작한 네트워크 구조를 활용하여 처음부터 훈련합니다.
네트워크 구조는 Model_Class_From_the_Scratch.py 파일을 통해 확인할 수 있습니다.

In [7]:
print("Learning from scratch")
training_class.prepare_network(True)
training_class.training_network(learning_rate = 0.0001, epochs=10, step_size=3, gamma=0.3)

Learning from scratch


100%|██████████| 148/148 [02:38<00:00,  1.07s/it]


[1] Test Loss: 1.2507, Accuracy: 47.98%
model saved!


100%|██████████| 148/148 [02:29<00:00,  1.01s/it]


[2] Test Loss: 1.2010, Accuracy: 53.38%
model saved!


...

[5] Test Loss: 1.1588, Accuracy: 56.90%
model saved!


100%|██████████| 148/148 [02:24<00:00,  1.03it/s]


[6] Test Loss: 1.1539, Accuracy: 57.62%
model saved!


 92%|█████████▏| 407/442 [09:21<00:48,  1.38s/it]


KeyboardInterrupt: 

다음은 ImageNet의 천 개 클래스의 백만 장 이미지로 학습한 MobileNetv2 네트워크를 전이학습합니다.
MobileNetv2 네트워크의 마지막 레이어의 출력 노드 수를 우리가 구분하고자 하는 클래스 수에 맞게 수정한 후 학습하며, 코드는 Model_Class_Transfer_Learning_MobileNet.py에서 확인할 수 있습니다.

In [ ]:
print("Transfer learning")
training_class.prepare_network(False)
training_class.training_network(learning_rate = 0.00001, epochs=5, step_size=3, gamma=0.3)

모든 학습이 끝나면 label_map.txt와 PyTorch_Training_From_Scratch.pt, PyTorch_Transfer_Learning_MobileNet.pt 파일을 다운로드합니다.

이후 Jetson 보드에서 해당 파일로 추론을 수행할 수 있습니다.